In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [3]:
spark = SparkSession.builder.appName("explore").getOrCreate()

24/10/08 22:22:32 WARN Utils: Your hostname, codespaces-cba268 resolves to a loopback address: 127.0.0.1; using 10.0.0.126 instead (on interface eth0)
24/10/08 22:22:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/08 22:22:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
import os

movie_tables = {}

for file in os.listdir("../data"):
    print(file.split(".")[:2])
    movie_tables["_".join(file.split(".")[:2])] = (
        spark.read.options(
            **{
                "sep": "\t",
                "header": True,
                "compression": "gzip",
                "nullValue": r"\N",
            }
        )
        .csv(f"../data/{file}")
        .limit(100)
    )
    # display(pd.read_csv(f"../data/{file}", compression = "gzip", sep = "\t", nrows = 10, na_values=r"\N").head(10))

['title', 'akas']


In [5]:
import transformation.bronze

In [6]:
bronze_transformation = {
    "name_basics": transformation.bronze.name_basics.transformation,
    "title_akas": transformation.bronze.title_akas.transformation,
    "title_basics": transformation.bronze.title_basics.transformation,
    "title_crew": transformation.bronze.title_crew.transformation,
    "title_episode": transformation.bronze.title_episode.transformation,
    "title_principals": transformation.bronze.title_principals.transformation,
    "title_ratings": transformation.bronze.title_ratings.transformation,
}

cleansed_movie_tables = {}

for _table_name, _sdf in movie_tables.items():
    cleansed_movie_tables[_table_name] = _sdf.transform(
        bronze_transformation[_table_name]
    )

AttributeError: module 'transformation.bronze' has no attribute 'title_ratings'

24/10/08 22:22:22 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [11]:
cleansed_movie_tables["name_basics"].show()

+---------+-------------------+---------+---------+--------------------+--------------------+
|   nconst|        primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+-------------------+---------+---------+--------------------+--------------------+
|nm0000001|       Fred Astaire|     1899|     1987|[actor,miscellane...|[tt0072308,tt0050...|
|nm0000002|      Lauren Bacall|     1924|     2014|[actress,soundtra...|[tt0037382,tt0075...|
|nm0000003|    Brigitte Bardot|     1934|     NULL|[actress,music_de...|[tt0057345,tt0049...|
|nm0000004|       John Belushi|     1949|     1982|[actor,writer,mus...|[tt0072562,tt0077...|
|nm0000005|     Ingmar Bergman|     1918|     2007|[writer,director,...|[tt0050986,tt0083...|
|nm0000006|     Ingrid Bergman|     1915|     1982|[actress,producer...|[tt0034583,tt0036...|
|nm0000007|    Humphrey Bogart|     1899|     1957|[actor,producer,m...|[tt0034583,tt0037...|
|nm0000008|      Marlon Brando|     1924|     2004|[actor,di

In [6]:
import pyspark_functions as F_

In [7]:
# F.split("genres", pattern=",").alias("genres"),
# F_.when_mapping(F.col("isAdult"), {1: True, 0: False}).alias("isAdult"),

24/10/08 22:18:30 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
